# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [39]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [40]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description			
"Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops"	Conference Presentation	ICA-2023	International Communication Association (ICA)	2023-05-27	"Toronto, ON, Canada"		"Hiaeshutter-Rice, D., Madrigal, G., Ploger, G., Carr, S., Carbone, M., <b> Battocchio, A.F.,</b> and Soroka, S. (2023). ""Identity Driven Information Ecosystems: TikTok, Truckstops, and Barbershops."" To be presented at the 73rd International Communication Association Conference, Toronto, Canada."			
Localizing social justice stories: Social media and local civic information infrastructure in six midwestern United States communities	Conference Presentation	AEJMC-2022-1	Communication Theory and Methods Division of the Association for Education in Journalism and Mass Communication(AEJMC)	2022-08-05	"Detroit, MI, USA"		"<b>Battocchio, A. F.,</b> Thorson, K., Hiaeshutter‑Rice, D., Smith, M., Chen, Y., Edgerly, S., Cotter, K., Denzin, K., 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [41]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Identity Driven Information Ecosystems: TikTok...,Conference Presentation,ICA-2023,International Communication Association (ICA),2023-05-27,"Toronto, ON, Canada",NaN,"Hiaeshutter-Rice, D., Madrigal, G., Ploger, G....",NaN,NaN,NaN
1,Localizing social justice stories: Social medi...,Conference Presentation,AEJMC-2022-1,Communication Theory and Methods Division of t...,2022-08-05,"Detroit, MI, USA",NaN,"<b>Battocchio, A. F.,</b> Thorson, K., Hiaeshu...",NaN,NaN,NaN
2,Discursive Opportunity Structure: The evolutio...,Conference Presentation,AEJMC-2022-2,Communication Theory and Methods Division of t...,2022-08-05,"Detroit, MI, USA",NaN,"Chen, Y., Choung, H., <b>Battocchio, A.F.</b>,...",NaN,NaN,NaN
3,Reaching the Rust Belt: Exploring identity and...,Conference Presentation,AEJMC-2022-3,Political Communication Division of the Associ...,2022-08-03,"Detroit, MI, USA",NaN,"<b>Battocchio, A.F.</b> and Smith, M. (2022). ...",NaN,NaN,NaN
4,Hyperlocal affective polarization: Remixing ru...,Conference Presentation,AEJMC-2021-1,Communication Theory and Methods Division of t...,2021-08-06,Virtual,NaN,"<b>Battocchio, A.F. </b>(2021) “Hyperlocal aff...",NaN,NaN,NaN
5,"“Do Good and Be ‘Liked'"": Corporate messaging ...",Conference Presentation,AEJMC-2021-2,Advertising Division of the Association for Ed...,2021-08-07,Virtual,NaN,"Yang, J., Bertellotti, E., <b>Battocchio, A.F....",NaN,NaN,NaN
6,A systematic method of cataloging civic inform...,Conference Presentation,AEJMC-2021-3,Communication Theory and Methods Division of t...,2021-08-05,Virtual,NaN,"<b>Battocchio, A. F.</b>, Etheridge, C., Thors...",NaN,NaN,NaN
7,Change is the only constant: Young adults as p...,Conference Presentation,AEJMC-2021-4,Mass Communication and Society Division of the...,2021-08-07,Virtual,NaN,"Thorson, K. and <b>Battocchio, A.F</b>. (2021)...",NaN,NaN,NaN
8,Effects of transparent brand communication on ...,Conference Presentation,AEJMC-2020-1,Advertising Division of the Association for Ed...,2020-08-07,Virtual,NaN,"Yang, J. and <b>Battocchio, A.F. </b>(2020). “...",NaN,NaN,NaN
9,Building brand authenticity on social media: T...,Conference Presentation,AEJMC-2020-2,Advertising Division of the Association for Ed...,2020-08-07,Virtual,NaN,"Yang, J., Teran, C., <b>Battocchio, A.F</b>., ...",NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [42]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [43]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [44]:
!ls ../_talks

2015-02-25-UMD-2015.md	    2021-05-03-CCST-2021.md
2015-03-05-UWS-2015-1.md    2021-06-09-CEC-2021.md
2015-03-05-UWS-2015-2.md    2021-08-05-AEJMC-2021-3.md
2016-02-28-NWO-2016.md	    2021-08-06-AEJMC-2021-1.md
2019-05-04-C4C-2019-1.md    2021-08-07-AEJMC-2021-2.md
2019-07-18-C4C-2019-4.md    2021-08-07-AEJMC-2021-4.md
2019-07-27-C4C-2019-2.md    2022-03-26-AAA-2022.md
2019-08-08-AEJMC-2019.md    2022-05-29-ICA-2022-2.md
2019-09-25-C4C-2019-3.md    2022-05-30-ICA-2022-1.md
2019-11-08-DEP-2019.md	    2022-08-03-AEJMC-2022-3.md
2020-01-22-C4C-2020-1.md    2022-08-04-AEJMC-2022-4.md
2020-06-01-C4C-2020-2.md    2022-08-05-AEJMC-2022-1.md
2020-08-07-AEJMC-2020-1.md  2022-08-05-AEJMC-2022-2.md
2020-08-07-AEJMC-2020-2.md  2022-11-19-NCA-2021.md
2021-01-22-C4C-2021.md	    2023-05-27-ICA-2023.md


In [45]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
